In [1]:
import numpy as np
import pandas as pd
import os
import sys
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# cleanse_pipe is a cleanse script I made for this project.
from cleanse_pipe import cleanse_data
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import math
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import time
import pycaret
from pycaret.classification import *

# This is my next stab at improving my model
from sklearn.ensemble import VotingClassifier

In [2]:
train = pd.read_csv('train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
data, passengers = cleanse_data(train)
# %%
data.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Alone,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Ms,Sex_Male,Farebin,Agebin
0,0,3,22.0,1,0,7.2500,0,0,1,0,0,0,1,0,0,1,2,2
1,1,1,38.0,1,0,71.2833,1,0,0,0,0,0,0,1,0,0,5,4
2,1,3,26.0,0,0,7.9250,0,0,1,1,0,1,0,0,0,0,2,2
3,1,1,35.0,1,0,53.1000,0,0,1,0,0,0,0,1,0,0,5,4
4,0,3,35.0,0,0,8.0500,0,0,1,1,0,0,1,0,0,1,2,4


In [4]:
# normalize data since we are going to be using models that need normalization
exp_clf = setup(data, target = 'Survived', train_size = .8, normalize= True, log_experiment= False)

,Description,Value
0,session_id,8839
1,Target,Survived
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(891, 18)"
5,Missing Values,False
6,Numeric Features,10
7,Categorical Features,7
8,Ordinal Features,False
9,High Cardinality Features,False


In [5]:
# exp_clf[0].head()

#%%

best = compare_models()
# %%


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8273,0.8781,0.7120,0.8057,0.7546,0.6224,0.6263,0.0190
catboost,CatBoost Classifier,0.8217,0.8705,0.6966,0.8036,0.7452,0.6092,0.6137,0.9680
lightgbm,Light Gradient Boosting Machine,0.8189,0.8600,0.7187,0.7830,0.7478,0.6070,0.6099,0.0140
ridge,Ridge Classifier,0.8146,0.0000,0.7340,0.7675,0.7472,0.6014,0.6048,0.0200
lda,Linear Discriminant Analysis,0.8146,0.8539,0.7342,0.7673,0.7472,0.6014,0.6048,0.0210
svm,SVM - Linear Kernel,0.8133,0.0000,0.7160,0.7759,0.7415,0.5962,0.6002,0.0230
ada,Ada Boost Classifier,0.8118,0.8465,0.7417,0.7571,0.7474,0.5977,0.5996,0.0170
lr,Logistic Regression,0.8103,0.8542,0.7157,0.7667,0.7394,0.5907,0.5924,0.2590
knn,K Neighbors Classifier,0.8020,0.8474,0.6852,0.7640,0.7212,0.5685,0.5716,0.1370
xgboost,Extreme Gradient Boosting,0.8006,0.8556,0.7004,0.7540,0.7251,0.5690,0.5711,0.0990


In [121]:
catboost = create_model('dt')


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8194,0.7988,0.7778,0.7500,0.7636,0.6176,0.6179
1,0.6944,0.6477,0.5556,0.6000,0.5769,0.3383,0.3390
2,0.8028,0.7949,0.6923,0.7500,0.7200,0.5682,0.5693
3,0.8310,0.8470,0.8462,0.7333,0.7857,0.6474,0.6519
4,0.7606,0.7256,0.6154,0.6957,0.6531,0.4713,0.4734
5,0.7887,0.7614,0.6667,0.7500,0.7059,0.5419,0.5442
6,0.7746,0.7441,0.6667,0.7200,0.6923,0.5149,0.5159
7,0.7887,0.7673,0.7407,0.7143,0.7273,0.5550,0.5552
8,0.7606,0.7353,0.6296,0.7083,0.6667,0.4809,0.4829
9,0.8028,0.7946,0.7778,0.7241,0.7500,0.5876,0.5886


In [122]:
tuned_cat = tune_model(catboost, n_iter= 100)



,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8611,0.9136,0.8148,0.8148,0.8148,0.7037,0.7037
1,0.7917,0.7930,0.6667,0.7500,0.7059,0.5455,0.5477
2,0.7887,0.8509,0.7308,0.7037,0.7170,0.5485,0.5488
3,0.8873,0.8936,0.8462,0.8462,0.8462,0.7573,0.7573
4,0.8592,0.9397,0.7308,0.8636,0.7917,0.6864,0.6919
5,0.8451,0.8843,0.7037,0.8636,0.7755,0.6591,0.6672
6,0.8169,0.7959,0.6667,0.8182,0.7347,0.5971,0.6044
7,0.7746,0.8624,0.7037,0.7037,0.7037,0.5219,0.5219
8,0.8310,0.8910,0.7407,0.8000,0.7692,0.6362,0.6374
9,0.8028,0.8439,0.8148,0.7097,0.7586,0.5933,0.5973


In [123]:
tuned_cat

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=10, max_features=1.0, max_leaf_nodes=None,
                       min_impurity_decrease=0.02, min_impurity_split=None,
                       min_samples_leaf=6, min_samples_split=9,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=8839, splitter='best')

In [94]:
test = pd.read_csv('test.csv')
test_data, passengers1 = cleanse_data(test)

preds = predict_model(tuned_cat, data = test_data)

ready_preds = pd.DataFrame(passengers1, columns = ['PassengerId'])
ready_preds['Survived'] = preds['Label']

ready_preds.to_csv('titanic_preds.csv', index = False)

In [14]:
data1 = data.copy()
target = data1['Survived']
del[data1['Survived']]

data1.head()

,Pclass,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Alone,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Ms,Sex_Male,Farebin,Agebin
0,3,22.0,1,0,7.2500,0,0,1,0,0,0,1,0,0,1,2,2
1,1,38.0,1,0,71.2833,1,0,0,0,0,0,0,1,0,0,5,4
2,3,26.0,0,0,7.9250,0,0,1,1,0,1,0,0,0,0,2,2
3,1,35.0,1,0,53.1000,0,0,1,0,0,0,0,1,0,0,5,4
4,3,35.0,0,0,8.0500,0,0,1,1,0,0,1,0,0,1,2,4


In [27]:
data1.describe()

,Pclass,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Alone,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Ms,Sex_Male,Farebin,Agebin
count,891.000000,8.910000e+02,891.000000,891.000000,8.910000e+02,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,-4.549298e-16,0.523008,0.381594,6.690620e-16,0.188552,0.086420,0.725028,0.602694,0.056117,0.204265,0.592593,0.140292,0.006734,0.647587,3.204265,3.041526
std,0.836071,1.000562e+00,1.102743,0.806057,1.000562e+00,0.391372,0.281141,0.446751,0.489615,0.230276,0.403390,0.491628,0.347485,0.081830,0.477990,1.208037,1.439984
min,1.000000,-2.221517e+00,0.000000,0.000000,-6.484217e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,2.000000,-6.075151e-01,0.000000,0.000000,-4.891482e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000
50%,3.000000,1.423457e-02,0.000000,0.000000,-3.573909e-01,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,3.000000,3.000000
75%,3.000000,4.600356e-01,1.000000,0.000000,-2.424635e-02,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,4.000000,4.000000
max,3.000000,3.793396e+00,8.000000,6.000000,9.667167e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,5.000000


In [26]:
# Standardize continuous variables

def standardScale_custom(x) :
    new_data = []
    stan_mean = np.mean(x)
    stan_std = np.std(x)
    for row in x :
        new_data.append((row - stan_mean) / stan_std)
    return new_data
    
for col in ['Age','Fare'] :
    data1[col] = standardScale_custom(data1[col])


In [133]:
x_train, x_test, y_train, y_test = train_test_split(data1, target, test_size=0.3, random_state= 0)

,Pclass,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Alone,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Ms,Sex_Male,Farebin,Agebin
331,1,1.185775,0,0,-0.074583,0,0,1,1,0,0,1,0,0,1,4,5
733,2,-0.514848,0,0,-0.386671,0,0,1,1,0,0,1,0,0,1,3,2
382,3,0.165401,0,0,-0.488854,0,0,1,1,0,0,1,0,0,1,2,3
704,3,-0.288098,1,0,-0.490280,0,0,1,0,0,0,1,0,0,1,2,2
813,3,-1.799763,4,2,-0.018709,0,0,1,0,0,1,0,0,0,0,4,1


In [134]:
svm = LinearSVC(penalty = 'l2', loss = 'squared_hinge', C = 5.0, fit_intercept= False, max_iter = 1000)

svm_fit = svm.fit(x_train, y_train)
svm_predict = svm_fit.predict(x_train)

print(classification_report(y_train, svm_predict))

              precision    recall  f1-score   support

           0       0.85      0.88      0.86       381
           1       0.80      0.75      0.77       242

    accuracy                           0.83       623
   macro avg       0.82      0.82      0.82       623
weighted avg       0.83      0.83      0.83       623



In [135]:
search_params = {"C" : [.01,.5,.8,1,2,5,7,10],
    "loss" : ['hinge', 'squared_hinge'],
    "fit_intercept" : [True,False]
}

grid = GridSearchCV(svm, param_grid= search_params, n_jobs = -1, cv = 10)

grid_fit = grid.fit(x_train, y_train)

print(grid_fit.best_params_)
print(grid_fit.best_score_)



{'C': 10, 'fit_intercept': False, 'loss': 'squared_hinge'}
0.8315412186379927


In [136]:
# start voting models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingClassifier

rf = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=11, max_features=1.0,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0001, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=110,
                       n_jobs=-1, oob_score=False, random_state=8839, verbose=0,
                       warm_start=False)

knn = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                     metric_params=None, n_jobs=-1, n_neighbors=20, p=2,
                     weights='uniform')
                    

lr = LogisticRegression(C=0.604, class_weight={}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=8839, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [137]:
voting = VotingClassifier(estimators = [('rf',rf),
    ('knn', knn),
    ('lr', lr)], voting = 'soft', n_jobs = -1)

voting_fit = voting.fit(x_train, y_train)
voting_predict = voting_fit.predict(x_test)

print(classification_report(y_test, voting_predict))

              precision    recall  f1-score   support

           0       0.86      0.87      0.86       168
           1       0.78      0.76      0.77       100

    accuracy                           0.83       268
   macro avg       0.82      0.81      0.82       268
weighted avg       0.83      0.83      0.83       268



In [138]:
test = pd.read_csv('test.csv')
test_data, passengers1 = cleanse_data(test)

for col in ['Age','Fare'] :
    test_data[col] = standardScale_custom(test_data[col])


ready_preds = pd.DataFrame(passengers1, columns = ['PassengerId'])
ready_preds['Survived'] = preds['Label']

ready_preds.to_csv('titanic_preds.csv', index = False)